In [ ]:
import pandas as pd

In [ ]:
path = '/content/'

In [ ]:
#connect to self drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive')


Mounted at /content/drive


In [ ]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
pip show gensim

Name: gensim
Version: 3.6.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /usr/local/lib/python3.7/dist-packages
Requires: smart-open, numpy, six, scipy
Required-by: 


# 1. online_shopping_10_cats.csv
data comes from :https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/online_shopping_10_cats/intro.ipynb
load data

In [ ]:

pd_all = pd.read_csv("/online_shopping_10_cats.csv")


# 
Extraction of data from specified categories

In [ ]:
target_cats = ['蒙牛', '水果'] 

pd_data = pd_all[pd_all.cat.isin(target_cats)]



pd_data.sample(20)

In [ ]:
pd_data = pd_data.dropna(subset=["review"])



In [ ]:
#save the balanced corpus
pd_data.to_csv(path+'fresh_food.csv',header=True,index=None)

In [ ]:
pd_use = pd.read_csv("/content/fresh_food.csv",index_col=None)
    
pos = pd_use[pd_use.label==1]
neg = pd_use[pd_use.label==0]
pos.to_csv(path+'pos.csv',header=True,index=None)

In [ ]:
neg.to_csv(path+'neg.csv',header=True,index=None)

# 2. word2vec Vectors training

In [ ]:
import numpy as np
import pandas as pd
import jieba
import multiprocessing
from gensim.models.word2vec import Word2Vec
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.decomposition import IncrementalPCA as PCA   
from sklearn.manifold import TSNE                  
import plotly.graph_objs as go
import seaborn as sns

import joblib

In [ ]:
   
    jieba.load_userdict("/content/drive/MyDrive/SogouLabDic.txt")
    jieba.load_userdict("/content/drive/MyDrive/dict_baidu_utf8.txt")
    jieba.load_userdict("/content/drive/MyDrive/dict_pangu.txt")
    jieba.load_userdict("/content/drive/MyDrive/dict_sougou_utf8.txt")
    jieba.load_userdict("/content/drive/MyDrive/dict_tencent_utf8.txt")
  

In [ ]:
#splits + remove stopwords
def cut_words(data):

    stopwords = {}.fromkeys([ line.rstrip() for line in open('/content/drive/MyDrive/cn_stopwords.txt','r',encoding='utf8') ])
    print(len(data))
    Result = []
    for item in data :
        #print(item)
        #print(type(item))
        seg = jieba.cut(item)
        res = [i for i in seg if i not in stopwords]
        Result.append(res)

    return Result

In [ ]:
def loadfile():
    pd_use = pd.read_csv("/content/drive/MyDrive/fresh_food.csv",index_col=None)
    
    pos = pd_use[pd_use.label==1]
    neg = pd_use[pd_use.label==0]
    #print(pos)
    
    #neg = pd.read_csv('./data/neg.csv', header=None, index_col=None)
    #pos = pd.read_csv('./data/pos.csv', header=None, index_col=None)

    combined = np.concatenate((pos["review"], neg["review"]))
    #combined = pd_use["review"]
    print(type(combined))
    y = np.concatenate((np.ones(len(pos), dtype=int), np.zeros(len(neg), dtype=int)))
    

    return combined, y

In [ ]:

def tokenizer(data):
    text = [jieba.lcut(document.replace('\n', '')) for document in data]
    return text


In [ ]:
# Adapted from Cyber Inovation Lab, github.com/das-lab/TAP/blob/master/tap.py
cpu_count = multiprocessing.cpu_count()
vocab_dim = 100
n_iterations = 1
n_exposures = 10  # All words with frequency above 10
window_size = 7


def word2vec_train(combined):
    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)  # input: list
    model.train(combined, total_examples=model.corpus_count, epochs=model.epochs)
    model.save('/content/drive/MyDrive/model/Word2vec_model_without.pkl')
   


# Word vectors summed to average
def fea_sentence(list_w):
    n0 = np.array([0. for i in range(vocab_dim)], dtype=np.float32)
    for i in list_w:
        n0 += i
    fe = n0 / len(list_w)
    fe = fe.tolist()
    return fe



In [ ]:
# train model and save
print('load data')
combined, y = loadfile()
print(len(combined))
print('data preparation')
combined = tokenizer(combined)
#combined = cut_words(combined)
print('train word2vec')
word2vec_train(combined)
## If the required word2vec has already been trained, the above lines of code can be commented out


In [ ]:
print('load word2vec model')
w2v= Word2Vec.load('/content/drive/MyDrive/model/Word2vec_model.pkl')


In [ ]:
print('load word2vec model')
w2v_without= Word2Vec.load('/content/drive/MyDrive/model/Word2vec_model_without.pkl')


# networks

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
pip show keras

In [ ]:
pip show tensorflow

In [ ]:
import codecs
import csv
import random

import jieba


import numpy as np
import pandas as pd

import multiprocessing
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras import Input, Model
from keras.preprocessing import sequence
from keras.models import load_model
from keras.layers import Conv1D, MaxPooling1D, concatenate
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

# Adapted from danan0755, GitHub.com/xyzgithub.com/danan0755/Bert_Classifier/blob/master/sentiment_predict.py
# Adapted from Cyber Inovation Lab, github.com/das-lab/TAP/blob/master/tap.py
cpu_count = multiprocessing.cpu_count()
vocab_dim = 100
n_iterations = 1
n_exposures = 10 # All words with frequency above 10
window_size = 7
n_epoch = 30
maxlen = 30
batch_size = 32



def create_dictionaries(model=None, combined=None):
  #w2indx - A dictionary that maps all words in a dataset into serial numbers
  #w2vec - A dictionary that maps all words in a dataset into vectors

    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k + 1 for k, v in gensim_dict.items()}  #Index of all words with frequency over 10, (k->v) => (v->k)



        w2vec = {word: model[word] for word in w2indx.keys()}  # # The word vector of all words with frequency over 10, (word->model(word)) 

        def parse_dataset1(combined):  
            data = []
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)  # freqxiao10->0
                data.append(new_txt)
            return data  # word=>index

        combined = parse_dataset1(combined)  # [[1,2,3...],[]]

 # The length of each sentence vector must be uniform, but the sentence length is not of a uniform form,
 # and a maxlen is set as the maximum sentence length value
 #The index corresponding to the word contained in each sentence, so sentences containing words with a frequency of less than 10 have an index of 0
        combined = sequence.pad_sequences(combined, maxlen=maxlen) 
        return w2indx, w2vec, combined
    else:
        print('No data provided...')



def use_word2vec_model(path,combined):
    model= Word2Vec.load(path)
    index_dict, word_vectors, combined = create_dictionaries(model=model, combined=combined)
    return index_dict, word_vectors, combined


def get_data_fortextcnn(index_dict, word_vectors, combined, y):
    n_symbols = len(index_dict) + 1  
    embedding_weights = np.zeros((n_symbols, vocab_dim))  
    for word, index in index_dict.items(): 
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2, random_state=5)
    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes=2)  # convert to one-hot 表示  [len(y),2]
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes=2)

    return n_symbols, embedding_weights, x_train, y_train, x_test, y_test


##define network
def train_cnn(n_symbols, embedding_weights, x_train, y_train):
    main_input = Input(shape=(maxlen,), dtype='float64')
    embedder = Embedding(output_dim=vocab_dim,
                         input_dim=n_symbols,
                         input_length=maxlen,
                         weights=[embedding_weights])
    embed = embedder(main_input)
    cnn1 = Conv1D(64, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=19)(cnn1)
    cnn2 = Conv1D(64, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=18)(cnn2)
    cnn3 = Conv1D(64, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=17)(cnn3)
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.5)(flat)
    main_output = Dense(2, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch)

    model.save('/content/drive/MyDrive/model/textcnn_w2v.h5')



In [ ]:

print('load word2vec ')
index_dict, word_vectors, combined = use_word2vec_model('/content/drive/MyDrive/model/Word2vec_model.pkl',combined)

print(' convert data format')
n_symbols, embedding_weights, x_train, y_train, x_test, y_test = get_data_fortextcnn(index_dict, word_vectors, combined,y)
print("features&labels size:")
print(x_train.shape, y_train.shape)

print('train textcnn')
train_cnn(n_symbols, embedding_weights, x_train, y_train)

print('load textcnn model')
model = load_model('/content/drive/MyDrive/model/textcnn_w2v.h5')
y_pred = model.predict(x_test)
for i in range(len(y_pred)):
    max_value = max(y_pred[i])
    for j in range(len(y_pred[i])):
        if max_value == y_pred[i][j]:
            y_pred[i][j] = 1
        else:
            y_pred[i][j] = 0
# target_names = ['neg', 'pos']
print(classification_report(y_test, y_pred))

Bi-LSTM

In [ ]:

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Bidirectional, Activation


In [ ]:



def train_bilstm(n_symbols, embedding_weights, x_train, y_train):
    print('Defining a Simple Keras Model...')
    model = Sequential()  # or Graph or whatever
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=n_symbols,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=maxlen))  # Adding Input Length
    # model.add(LSTM(output_dim=50, activation='tanh'))
    model.add(Bidirectional(LSTM(64, activation='tanh')))
   # model.add(LSTM(64, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # Dense=>Fully connected layer, output dimension = 2

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch)

    model.save('/content/drive/MyDrive/model/bilstm_w2v.h5')




In [ ]:


    print('train bilstm model')
    train_bilstm(n_symbols, embedding_weights, x_train, y_train)

    print('load bilstm model')
    model = load_model('/content/drive/MyDrive/model/bilstm_w2v.h5')
    y_pred = model.predict(x_test)

    for i in range(len(y_pred)):
        max_value = max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value == y_pred[i][j]:
                y_pred[i][j] = 1
            else:
                y_pred[i][j] = 0
    print(classification_report(y_test, y_pred))

cnnbilstm

In [ ]:

from keras.layers import Conv1D, MaxPooling1D, concatenate
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping

def train_cnn_bilstm(n_symbols, embedding_weights, x_train, y_train):
    #  Model structure: word embedding - convolutional pooling - splicing - BiLSTM - fully connected - dropout - fully connected
    main_input = Input(shape=(maxlen,), dtype='float64')
    embedder = Embedding(output_dim=vocab_dim,
                         input_dim=n_symbols,
                         input_length=maxlen,
                         weights=[embedding_weights])
    embed = embedder(main_input)
    cnn = Conv1D(64, 3, padding='same', strides=1, activation='relu')(embed)
    bilstm = Bidirectional(LSTM(64, dropout=0.5, activation='tanh', return_sequences=True))(cnn)
    flat = Flatten()(bilstm)
    main_output = Dense(2, activation='softmax')(flat)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch)

    model.save('/content/drive/MyDrive/model/cnnbilstm.h5')




In [ ]:
    print('train cnn_bilstm')
    train_cnn_bilstm(n_symbols, embedding_weights, x_train, y_train)

    print('load cnn_bilstm')
    model = load_model('/content/drive/MyDrive/model/cnnbilstm.h5')
    y_pred = model.predict(x_test)
    for i in range(len(y_pred)):
        max_value = max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value == y_pred[i][j]:
                y_pred[i][j] = 1
            else:
                y_pred[i][j] = 0
    print(classification_report(y_test, y_pred))

# 4.topic extraction

tf-idf

In [ ]:
pip show jieba

In [ ]:
import os
import jieba
import pandas as pd

import numpy as np
import jieba.analyse

from jieba import analyse


In [ ]:
#Get word vector word lists
def get_model_voca(model):
   words = model.wv.index2word
   #words = model.wv.index_to_key
   vocablist = []
   for word in words:  
     vocablist.append(word)
   return vocablist

In [ ]:
#Get similar words
def get_sims_words(word,vocablist,model):

  if word  in vocablist:
    sims = model.wv.most_similar(word, topn=10)  # get other similar words
    simslist = []

    i=0
    for i in range(0,len(sims)):
      voca = sims[i][0]
      simslist.append(voca)
      i=i+1
  
  else:
    simslist = []
  
  return simslist

In [ ]:
#Get topic
def get_attributes(keywordsdic):
   empty=[]
   for k in keywordsdic.keys():
     empty.append(k)
   return empty


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/fresh_food.csv', header=0, encoding='utf-8-sig')['review']
#remove stopwords
#tfidf = analyse.extract_tags
jieba.analyse.set_stop_words('/content/drive/MyDrive/cn_stopwords.txt')
for line in data:
    #Exclusion of empty lines
    if line.strip() == '':
        
        continue

    text = line
        #tfidf starts with statistical information about words only and does not sufficiently consider the semantic information between words
       
    keywords = tfidf(text,
                         allowPOS=('ns', 'nr', 'nt', 'nz', 'nl', 'n', 'vn', 'vd', 'vg', 'v', 'vf', 'a', 'an', 'i'))
    result = []

    for keyword in keywords:
        result.append(keyword)
    fo = open('/content/drive/MyDrive/keywords_withoutstop.txt', "a+", encoding='utf-8')  # generate keywords for every review
    for j in result:
        fo.write(j)
        fo.write(' ')
    fo.write('\n')
    fo.close()

kwords = ''
f = open('/content/drive/MyDrive/keywords_withoutstop.txt', 'r', encoding='utf-8')

for i in f:
    kwords += f.read()


    # TextRank
result = jieba.analyse.textrank(kwords, topK=50, withWeight=True)

keywords = dict()
for i in result:
    keywords[i[0]] = i[1]
print(keywords)





In [ ]:

filename = open('/content/drive/MyDrive/whole_corpus_withstopwords.txt','w')#dict to txt
for k,v in keywords.items():
    filename.write(k+':'+str(v))
    filename.write('\n')
filename.close()

In [ ]:


vectorwords = get_model_voca(w2v)
#change word2vec version
#vectorwords = get_model_voca(w2v_without)
attributes = get_attributes(keywords)

attri_result= dict()
topics =[]

for attribute in attributes:
  samiliarity  = get_sims_words(attribute,vectorwords,w2v)
  #topics for coherance
  topics.append(samiliarity)
  attri_result[attribute] = samiliarity


filename = open('/content/drive/MyDrive/attributs_withstop_w2v.txt','w')#dict to txt
for k,v in attri_result.items():
    filename.write(k+':'+str(v))
    filename.write('\n')
filename.close()




In [ ]:
print(topics)

topic coherence metrics

In [ ]:
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
def loadfiletest():
    pd_use = pd.read_csv("/content/drive/MyDrive/fresh_food.csv",index_col=None)
    
    pos = pd_use[pd_use.label==1]
    neg = pd_use[pd_use.label==0]
    pos2=pos.sample( frac=0.2)
    neg2=neg.sample( frac=0.2)
    #print(pos)
    
    #neg = pd.read_csv('./data/neg.csv', header=None, index_col=None)
    #pos = pd.read_csv('./data/pos.csv', header=None, index_col=None)

    combined = np.concatenate((pos2["review"], neg2["review"]))
    #combined = pd_use["review"]
   
    y = np.concatenate((np.ones(len(pos), dtype=int), np.zeros(len(neg), dtype=int)))
    

    return combined, y


In [ ]:
print('load test data')
combined, y = loadfiletest()
print(len(combined))
print('data preparation')
texts= tokenizer(combined)
#texts= cut_words(combined)

In [ ]:
word2id = Dictionary( texts )

In [ ]:


# Coherence model
#topics-attributes

cm1 = CoherenceModel(topics=topics, 
                    texts=texts,
                    coherence='c_v',  
                    dictionary=word2id)

cm2 = CoherenceModel(topics=topics, 
                    texts=texts,
                    coherence='u_mass',  
                    dictionary=word2id)

coherence1 = cm1.get_coherence()

coherence2 = cm2.get_coherence()


In [ ]:
print(coherence1)
print(coherence2)